Articulation features:

1)Cepstral Peak Prominence  | Lower in PD than in healthy

2)Vowel Space Area  |  Lower in PD than in healthy	

3)Vowel Articulation Index  |  	Lower in PD than in healthy	


Cepstral Peak Prominence

First I will start by defining a cepstrum. Consider we have a sound signal x[n].

The following are the stepa required to generate the cepstrum:

1) Take fourier transform X(f)
2) Take log(|X(f)|)
3) Take Fourier transform agin of result of part 2 and get C(q)  (C: cepstrum ; q:quefrency (has units of time))

The cepstral peak was defined as the highest amplitude peak in the cepstrum, and CPP is defined as the distance from cepstral peak to a linear regression line showing the average energy of a sound.(1) This definition is applied for each frame. Therefore, the values we are going to extract are the mean of the CPP values for all the frames of the recording. We are going to get the standard deviation of the distribution of the values of CPP.





In [1]:
import os
import glob
import numpy as np
import pandas as pd
import librosa

# -------- CPP FUNCTIONS --------

def compute_cpp_from_signal(y, sr, fmin=60.0, fmax=300.0):
    # take ~1 second from the middle
    if len(y) > sr:
        center = len(y) // 2
        half = sr // 2
        y = y[center - half : center + half]

    window = np.hanning(len(y))
    y_win = y * window

    spectrum = np.abs(np.fft.rfft(y_win)) ** 2
    log_power = 10 * np.log10(spectrum + 1e-12)

    cepstrum = np.fft.irfft(log_power)
    q = np.arange(len(cepstrum)) / sr

    min_q = 1.0 / fmax
    max_q = 1.0 / fmin
    idx = np.where((q >= min_q) & (q <= max_q))[0]

    region = cepstrum[idx]
    peak_global_idx = idx[np.argmax(region)]
    peak_val = cepstrum[peak_global_idx]

    x = q[idx]
    A = np.vstack([x, np.ones_like(x)]).T
    m, b = np.linalg.lstsq(A, cepstrum[idx], rcond=None)[0]
    baseline_at_peak = m * q[peak_global_idx] + b

    cpp_db = float(peak_val - baseline_at_peak)
    return cpp_db

def compute_cpp_for_file(path):
    y, sr = librosa.load(path, sr=None)
    return compute_cpp_from_signal(y, sr)

# -------- LOOP OVER FILES --------

audio_folder = r"C:\Users\user\Downloads\archive\26_29_09_2017_KCL\26-29_09_2017_KCL\SpontaneousDialogue\PD"  # folder with .wav

wav_files = sorted(glob.glob(os.path.join(audio_folder, "*.wav")))
print(f"Found {len(wav_files)} wav files")

rows = []
for wav in wav_files:
    try:
        cpp = compute_cpp_for_file(wav)
    except Exception as e:
        print(f"Error processing {wav}: {e}")
        cpp = np.nan

    rows.append({
        "file_name": os.path.basename(wav),
        "cpp_db": cpp
    })

df_cpp = pd.DataFrame(rows)

# ✅ save CSV in the SAME FOLDER as the Jupyter notebook
notebook_folder = os.getcwd()
out_csv = os.path.join(notebook_folder, "cpp_features.csv")
df_cpp.to_csv(out_csv, index=False)

print("Saved CPP features to:", out_csv)
df_cpp.head()


Found 15 wav files
Saved CPP features to: c:\Users\user\Desktop\490\490 project\cpp_features.csv


,file_name,cpp_db
0,ID02_pd_2_0_0.wav,0.277514
1,ID04_pd_2_0_1.wav,0.174915
2,ID06_pd_3_1_1.wav,0.133030
3,ID07_pd_2_0_0.wav,0.350973
4,ID13_pd_3_2_2.wav,0.375935


Vowel Space Area and Vowel articulation index are important acoustic features for parkinson disease detection but they require specific recordings of people pronouncing some vowels. Unfortunately, there are no accessible datasets that has those specific voice recordings. 

So we can consider one of the futur work that can be done is gathering the needed datasets or ask authors of some research papers to give us access to their datasets.

References:

1) Efficacy of Cepstral Measures in Voice Disorder Diagnosis: A Literature Review 

        Authors: Mahshid Aghajanzadeh , Saeed Saeedi
       